INGESTÃO DE DADOS


In [2]:
import pandas as pd
import os

file_paths = {
    'admissions': 'rawcsvs/ADMISSIONS.csv',
    'microbiology_events': 'rawcsvs/MICROBIOLOGYEVENTS.csv',
    'prescriptions': 'rawcsvs/PRESCRIPTIONS.csv',
    'labevents': 'rawcsvs/LABEVENTS.csv',
    'diagnoses_icd': 'rawcsvs/DIAGNOSES_ICD.csv',
    'patients': 'rawcsvs/PATIENTS.csv',
    'transl-labitems': 'rawcsvs/D_LABITEMS.csv',
    'transl-diag': 'rawcsvs/D_ICD_DIAGNOSES.csv'
}

cols_to_use = {
    'admissions': ['subject_id','admittime','ethnicity', 'admission_type'],
    'microbiology_events': ['row_id', 'subject_id',  'chartdate', 'charttime', 'spec_itemid', 'spec_type_desc', 
                            'org_itemid', 'org_name', 'isolate_num', 'ab_itemid', 'ab_name', 'dilution_text', 
                            'dilution_comparison', 'dilution_value', 'interpretation'],
    'prescriptions': ['subject_id', "hadm_id", 'drug', 'enddate'],
    'labevents': ['subject_id', 'itemid', 'valuenum', 'valueuom', 'flag'],
    'diagnoses_icd': ['subject_id', 'icd9_code'], #gotta work on this one, adding all codes to one line in the array
    'transl-labitems': ['itemid', 'label'], # use this in conjunction with labevents to understando wtf if happening
    'patients': ['subject_id', 'gender'],
    'transl-diag': ['icd9_code', 'short_title', 'long_title']
}

files = {}
for file_path in file_paths:
    files[file_path] = pd.read_csv(file_paths[file_path], usecols=cols_to_use[file_path])
    
for file_path, file in files.items(): 
    if  not file_path.startswith('transl'):
        file.dropna(inplace=True, subset=['subject_id'])
        file.drop_duplicates(inplace=True)  
        file_name = os.path.basename(file_path)
        file.to_csv(f'cleanedcsv/{file_name}.csv', index=False)

MANEIRA PARA JUNTAR TODOS OS DADOS


In [3]:
'''
Isso esta feito para essas duas tabelas
o modo nao mudou, precisa mudar as tabelas importadas, as colunas pivoteadas, e o valor de value, dependendo do que 
precisa ser pivoteado
Agradeco Pedro por me dar a idea de pivotear as tabelas
'''
import pandas as pd
mergedata = pd.read_csv('mergedcsv/admin-diag-lab-micro.csv')
diagnoses = pd.read_csv('cleanedcsv/prescriptions.csv')


#inverter a tabela de diagnósticos para ter uma linha por paciente

diaginv = diagnoses.assign(value=1).pivot_table(
    index='subject_id', 
    columns='drug', 
    values='value', 
    fill_value=0
).reset_index()    # Reset the index to make `subject_id` a column

diaginv.fillna(0, inplace=True)

# Now proceed with the merge
mergedata = mergedata.merge(diaginv, on='subject_id', how='left')




TRADUCAO DE CODIGOS PARA TEXTOS


In [ ]:
import pandas as pd
import os
#Need to convert the icd9_codes into text
def tryconvertint(possibleint):
    try:
        x = int(possibleint)
        return x
    except ValueError:
        return possibleint 

#first, I will get a representation of all the possible codes and their respective values
unique_diag_codes = pd.read_csv(os.path.join('rawcsvs', 'D_ICD_DIAGNOSES.csv')).filter(['icd9_code', 'short_title']).drop_duplicates()
unique_labitem_codes = pd.read_csv(os.path.join('rawcsvs', 'D_LABITEMS.csv')).filter(['itemid', 'label']).drop_duplicates()

#second, I will iterate through each column name and replace it with the title and/or label
allthedata = pd.read_csv(os.path.join('mergedcsv', 'admin-diag-lab-micro-pat-prescrip.csv'))

new_columns = []

for column in allthedata.columns:
    colint = tryconvertint(column)
    if colint == 50800:
        x = unique_labitem_codes['itemid'].values
        pass

    # Check if the column is in the 'icd9_code' of unique_diag_codes
    if column in unique_diag_codes['icd9_code'].values:
        # Map it to 'short_title'
        new_columns.append(unique_diag_codes.loc[unique_diag_codes['icd9_code'] == column, 'short_title'].values[0])

    # Check if the column is in the 'itemid' of unique_labitem_codes
    elif colint in unique_labitem_codes['itemid'].values:
        # Map it to 'label'
        new_columns.append(unique_labitem_codes.loc[unique_labitem_codes['itemid'] == colint, 'label'].values[0])

    # If not found, retain the original column name
    else:
        new_columns.append(column)

allthedata.columns = new_columns
allthedata.columns


EXTRACAO DE DADOS IMPORTANTES A BUSCA


In [11]:
import Levenshtein
#supunhetemos que tenhamos uma busca
ab_name = 'vancomycin'  # Define a default value for ab_name
search = ab_name if ab_name else 'vancomycin'
#precisamos listar todas as doencas em que o medicamento foi usado
#para facilitar a busca, precisamos calcular o medicamento mais provavel que foi escrito
all_drugs = files['microbiology_events']['ab_name'].dropna().unique()

closest_drug = None	
for drug in all_drugs:
    dist = Levenshtein.distance(search, drug)
    if closest_drug is None or dist < closest_drug[1]:
        closest_drug = (drug, dist)

#agora que temos o medicamento mais provavel, podemos listar as resistencias

Coletando os dados de resistencia


In [12]:
import plotly.express as px

#need to sort he diseases
closest_drug = files['microbiology_events']['ab_name'].mode()[0]
diseases = files['microbiology_events'][files['microbiology_events']['ab_name'] == closest_drug[0]]['org_name'].unique()



dis_res_count = {}
for disease in diseases:
    disease_res = files['microbiology_events'][
        (files['microbiology_events']['ab_name'] == closest_drug[0]) & 
        (files['microbiology_events']['org_name'] == disease)
    ]['interpretation']
    
    # Initialize counts for 'R', 'I', 'S'
    counts = {'R': 0, 'I': 0, 'S': 0}
    
    counts.update(disease_res.value_counts().to_dict())
    
    dis_res_count[disease] = counts





FODASSE O MODELO PREDITIVO


In [ ]:
from flask import Flask, render_template, request, jsonify, redirect, url_for, session
import Levenshtein

app = Flask(__name__)
app.secret_key = 'VAI SE FUDER FIGA' #perdao bita cabrita

@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'GET':
        return render_template('busca.html')
    elif request.method == 'POST':
        ab_name = request.form.get('ab_name')

        def search(term):
            all_drugs = files['microbiology_events']['ab_name'].dropna().unique()

            closest_drug = None    
            for drug in all_drugs:
                dist = Levenshtein.distance(term, drug)
                if closest_drug is None or dist < closest_drug[1]:
                    closest_drug = (drug, dist)
            return closest_drug
        

        if ab_name:
            droga = search(ab_name)
            session['droga'] = droga[0]  # Armazena apenas o nome da droga na sessão
            return redirect(url_for('resposta'))
        else:
            return jsonify({"error": "Nome do antibiótico não informado"}), 400

# Rota para a página secundaria (resposta)

@app.route('/resposta')

def resposta():
    closest_drug = session.get('droga')  # Recupera a droga da sessão

    # Verifica se o medicamento foi encontrado
    if closest_drug:
        # Chama a função que calcula as contagens de resistência
        resistencias = dis_res_count(closest_drug)
    else:
        resistencias = {}

    # Renderiza o template com os dados de resistência
    return render_template('resposta.html', closest_drug=closest_drug, resistencias=resistencias)

if __name__ == '__main__':
    app.run() 


def dis_res_count(closest_drug):
    # Filtra as doenças baseadas no medicamento mais próximo
    diseases = files['microbiology_events'][files['microbiology_events']['ab_name'] == closest_drug]['org_name'].unique()
    
    # Dicionário para armazenar a contagem de resistência por doença
    dis_res_count = {}

    # Itera sobre cada doença e conta as interpretações 'R', 'I', 'S'
    for disease in diseases:
        disease_res = files['microbiology_events'][
            (files['microbiology_events']['ab_name'] == closest_drug) & 
            (files['microbiology_events']['org_name'] == disease)
        ]['interpretation']
        
        # Inicializa a contagem de 'R', 'I', 'S'
        counts = {'R': 0, 'I': 0, 'S': 0}
        
        # Atualiza as contagens com base nas interpretações
        counts.update(disease_res.value_counts().to_dict())
        
        # Armazena a contagem da doença atual
        dis_res_count[disease] = counts

    return dis_res_count


Criando o dataset para o Modelo

In [1]:
import pandas as pd
import os 

patients = pd.read_csv(os.path.join('rawcsvs', 'PATIENTS.csv'))
admissions = pd.read_csv(os.path.join('rawcsvs', 'ADMISSIONS.csv'))
microbiology = pd.read_csv(os.path.join('rawcsvs', 'MICROBIOLOGYEVENTS.csv'))


# Merge the patients and admissions dataframes  
new_data = (admissions
            .filter(['subject_id', 'admission_type', 'ethnicity', 'diagnosis', 'hospital_expire_flag', 'insurance', 'religion', 'marital_status'])
            #.drop(admissions[admissions['hospital_expire_flag'] == 1].index)
            .merge(patients, on='subject_id', how='left')
            )
#merge the microbiology data
new_data = (
    microbiology
    .filter(['subject_id', 'ab_name', 'interpretation'])
    # One hot encoding the interpretation
    .assign(value=(microbiology['interpretation'] == 'R').astype(int))
    .reset_index(drop=True)
    .merge(new_data, on='subject_id', how='left')
)



aux1 = ( # one hot enconding e passando para formato largo os diagnósticos
    new_data
    .filter(['subject_id', 'diagnosis'])
    .assign(value=1)
    .pivot_table(index='subject_id', columns='diagnosis', values='value', fill_value=0)
    .reset_index()
)
aux2 = (
    new_data
    .groupby('subject_id')
    .agg({
        'insurance': 'first',
        'religion': 'first',
        'marital_status': 'first',
        'ethnicity': 'first',
        'gender': 'first',
        'ab_name': 'first',
        'interpretation': 'first'
    })
    .reset_index()
)

filtered_data = aux1.merge(aux2, on='subject_id', how='left')
filtered_data.to_csv('improving.csv', index=False)





Feature Engineering (Fuck data science)


In [ ]:
import pandas as pd
import re

# Assuming your DataFrame is called 'df'
df = pd.read_csv('improving.csv', sep=',')

# Step 1: Identify non-disease columns
non_disease_columns = ['subject_id', 'insurance', 'religion', 'marital_status', 'ethnicity', 'gender', 'ab_name', 'interpretation'] 

# Identify disease columns by excluding non-disease columns
disease_columns = [col for col in df.columns if col not in non_disease_columns]

# Step 2: Create a mapping from each disease to disease types
# Define your disease types and map diseases to them
disease_type_mapping = {
    'Cardiovascular': ['ACUTE PULMONARY EMBOLISM', 'CONGESTIVE HEART FAILURE', 'CORONARY ARTERY DISEASE', 'STEMI', 'MI CHF', 'BRADYCARDIA', 'VF ARREST', 'PERICARDIAL EFFUSION', 'PULMONARY EDEMA, MI', 'PULMONARY EDEMA\\CATH', 'CRITICAL AORTIC STENOSIS/HYPOTENSION', 'INFERIOR MYOCARDIAL INFARCTION\\CATH'],
    'Respiratory': ['ACUTE RESPIRATORY DISTRESS SYNDROME', 'ASTHMA', 'COPD FLARE', 'PNEUMONIA', 'RESPIRATORY DISTRESS', 'SHORTNESS OF BREATH', 'HYPOXIA', 'TACHYPNEA', 'PLEURAL EFFUSION', 'MEDIASTINAL ADENOPATHY'],
    'Neurological': ['ALTERED MENTAL STATUS', 'SEIZURE', 'STATUS EPILEPTICUS', 'STROKE/TIA', 'INTRACRANIAL HEMORRHAGE', 'HEADACHE', 'FACIAL NUMBNESS', 'BASAL GANGLIN BLEED', 'ALTERED MENTAL STATUS'],
    'Infectious': ['CELLULITIS', 'FEVER', 'SEPSIS', 'URINARY TRACT INFECTION', 'UROSEPSIS', 'PYELONEPHRITIS', 'UTI', 'HIV', 'ABSCESS', 'PNEUMONIA;TELEMETRY', 'SEPSIS; UTI', 'FEVER;URINARY TRACT INFECTION'],
    'Gastrointestinal': ['ABDOMINAL PAIN', 'GASTROINTESTINAL BLEED', 'LOWER GI BLEED', 'UPPER GI BLEED', 'VARICEAL BLEED', 'HEPATITIS', 'LIVER FAILURE', 'CHOLECYSTITIS', 'CHOLANGITIS', 'PANCREATITIS', 'ACUTE CHOLECYSTITIS', 'ACUTE CHOLANGITIS', 'HEPATITIS B', 'ALCOHOLIC HEPATITIS', 'ELEVATED LIVER FUNCTIONS;S/P LIVER TRANSPLANT', 'FAILURE TO THRIVE'],
    'Renal': ['ACUTE RENAL FAILURE', 'CHRONIC KIDNEY DISEASE', 'RENAL CANCER', 'HYPOTENSION, RENAL FAILURE'],
    'Endocrine': ['DIABETES', 'HYPOGLYCEMIA', 'HYPERGLYCEMIA', 'HYPONATREMIA'],
    'Oncology': ['LUNG CANCER', 'BREAST CANCER', 'COLON CANCER', 'LEUKEMIA', 'LYMPHOMA', 'METASTATIC MELANOMA', 'NON SMALL CELL CANCER', 'METASTIC MELANOMA;ANEMIA', 'CHRONIC MYELOGENOUS LEUKEMIA;TRANSFUSION REACTION'],
    'Trauma': ['FRACTURE', 'MOTOR VEHICLE ACCIDENT', 'SUBDURAL HEMATOMA', 'LEFT HIP FRACTURE', 'RIGHT HUMEROUS FRACTURE', 'HUMERAL FRACTURE', 'S/P MOTOR VEHICLE ACCIDENT', 'ACUTE SUBDURAL HEMATOMA'],
    'Other': ['OVERDOSE', 'FAILURE TO THRIVE', 'MEDIASTINAL ADENOPATHY', 'ARREST', 'OVERDOSE', 'AROMEGLEY;BURKITTS LYMPHOMA', 'TRACHEAL ESOPHAGEAL FISTULA', 'TRACHEAL STENOSIS', 'VOLVULUS', 'ANEMIA', 'BURKITTS LYMPHOMA', 'NON SMALL CELL CANCER;HYPOXIA', 'METASTATIC MELANOMA;BRAIN METASTASIS']
}

# Initialize a dictionary to map each disease column to its disease type(s)
disease_to_type = {}

# Process each disease column
for col in disease_columns:
    # Split disease names if they contain multiple diseases
    diseases = re.split(';|,|/', col)
    diseases = [disease.strip().upper() for disease in diseases]
    types = set()
    for disease in diseases:
        # Find the disease type for this disease
        found = False
        for disease_type, disease_list in disease_type_mapping.items():
            if disease in disease_list:
                types.add(disease_type)
                found = True
        if not found:
            types.add('Other')  # Assign 'Other' if disease not found in mapping
    disease_to_type[col] = types

# Step 3: For each disease type, sum the corresponding disease columns
for disease_type in disease_type_mapping.keys():
    # Get all columns that map to this disease type
    cols = [col for col, types in disease_to_type.items() if disease_type in types]
    if cols:
        # Sum the columns (since they are binary, this counts the number of diseases)
        df[disease_type] = df[cols].sum(axis=1)
    else:
        df[disease_type] = 0  # If no diseases of this type are present

# Step 4: Optionally, drop the individual disease columns
df.drop(columns=disease_columns, inplace=True)

df.to_csv('improving2.csv', index=False)

# Now, df contains the non-disease columns and the summed disease type columns
print(df.head())


Testando Melhores Variaveis

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Update the classifier in the pipeline to Gradient Boosting
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(random_state=42))
])

# Define parameter grid for Gradient Boosting tuning
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [3, 5, 10],
    'classifier__learning_rate': [0.01, 0.1, 0.2]
}

# Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(model_pipeline, param_grid, cv=3, n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(X_train, y_train)

# Evaluate the model
y_pred = grid_search.best_estimator_.predict(X_test)
print('Classification Report:\n', classification_report(y_test, y_pred))


In [ ]:
import joblib



# Load the model later
loaded_model = joblib.load('antibiotic_resistance_model.pkl')

'''
# Assuming new_data is a DataFrame with the same structure as your original X
new_predictions = loaded_model.predict(new_data)

# View the predictions
print(new_predictions)
'''
X_train.columns
